<h2 align="center">Собираем Датасет на основе музыки Spotify</h2> 

In [633]:
import json
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import librosa as lr
import pydub
import math
from sklearn.decomposition import PCA



In [634]:
path = 'Path to songs Spotify'
genres = os.listdir(path)
del genres[1]
path2 = 'Path to songs GTZAN'
genresgt = os.listdir(path2)
del genresgt[1]


Посмотрим сколько песен мы можем загрузить по длительности и количеству(возможно потом это будет важно)

In [640]:
msmin_bygenre = dict(list(zip(genres,[0 for _ in range(10)])))
numsong = list()
for genre in genres:
    songs = os.listdir(path+f'/{genre}')
    numsong.append(len(songs))
minsong = min(numsong)
numsong

[46, 43, 40, 45, 48, 48, 46, 50, 46, 47]

In [4]:
msmin_bygenregt = dict(list(zip(genresgt,[0 for _ in range(10)])))
for genre in genresgt:
    songs = os.listdir(path2+f'/{genre}')
    dlit = list()
    for j,s in tqdm(enumerate(songs)):
        try:
            dur = pydub.utils.mediainfo(path2+f'/{genre}'+f'/{s}')['duration']
            data, sr = lr.load(path2+f'/{genre}'+f'/{s}',  duration = math.floor(float(dur)))
        except:
            continue
        dlit.append(data.shape[0])
    msmin_bygenregt[genre] = min(dlit)

100it [00:15,  6.39it/s]
100it [00:13,  7.19it/s]
100it [00:14,  6.74it/s]
100it [00:14,  6.90it/s]
100it [00:14,  6.90it/s]
100it [00:14,  6.95it/s]
100it [00:14,  6.93it/s]
100it [00:14,  7.13it/s]
100it [00:14,  6.90it/s]
100it [00:14,  6.76it/s]


In [636]:
msmin_bygenregt

{'pop': 661500,
 'metal': 661500,
 'disco': 639450,
 'blues': 661500,
 'reggae': 661500,
 'classical': 639450,
 'rock': 639450,
 'hiphop': 639450,
 'country': 639450,
 'jazz': 661500}

In [638]:
for genre in genres:
    songs = os.listdir(path+f'/{genre}')
    dlit = list()
    for j,s in tqdm(enumerate(songs)):
        if j == minsong:
            break
        try:
            dur = pydub.utils.mediainfo(path+f'/{genre}'+f'/{s}')['duration']
            data, sr = lr.load(path+f'/{genre}'+f'/{s}',  duration = math.floor(float(dur)))
        except:
            continue
        dlit.append(data.shape[0])
    msmin_bygenre[genre] = min(dlit)

40it [00:29,  1.34it/s]
40it [00:29,  1.35it/s]
40it [00:25,  1.55it/s]
40it [00:32,  1.23it/s]
40it [00:30,  1.33it/s]
40it [00:25,  1.60it/s]
40it [00:29,  1.35it/s]
40it [00:28,  1.39it/s]
40it [00:26,  1.52it/s]
40it [00:35,  1.14it/s]


In [639]:
msmin_bygenre

{'reggae_spotify': 3351600,
 'disco_spotify': 3814650,
 'blues_spotify': 2579850,
 'metal_spotify': 3858750,
 'rock_spotify': 3219300,
 'pop_spotify': 617400,
 'hip hop_spotify': 3550050,
 'classical_spotify': 1278900,
 'country_spotify': 3351600,
 'jazz_spotify': 2998800}

#### Загружаем песни из датасета GTZAN
**GTZAN**
* Создан в ~2000 году
* Состоит из 10 жанров по 100 песен, длительностью 30 секунд
* Песня начинается с произвольного момента(однозначно не начало)
* При прослушивании некоторых песен что-то режет слух; Возможно в песнях есть какой-то шум, битый звук
* Непонятно, как собирались данные, нет кода по сбору\

Из всех предпосылок выше был сделан вывод, что этот датасет не стоит использовать. Однако код по сбору признаков остался.

In [396]:
gtzan_genre = dict(list(zip(genresgt,[0 for _ in range(10)])))
for genre in genresgt:
    ans = np.zeros((55,))
    songs = os.listdir(path2+f'/{genre}')
    print(genre)
    for j,s in tqdm(enumerate(songs)):
        if s == '.DS_Store':
            pass
        durts = pydub.utils.mediainfo(path2+f'/{genre}'+f'/{s}')['duration_ts']
        datak, sr = lr.load(path2+f'/{genre}'+f'/{s}',  duration = math.floor(float(durts)))
        for i in range(10):
            data = datak[i*66149:66149*(i+1)]
            mfcc = lr.feature.mfcc(y=data,sr=sr)
            stft = lr.feature.chroma_stft(y=data, sr=sr)
            rms = lr.feature.rms(y=data)
            centroid = lr.feature.spectral_centroid(y=data, sr=sr)
            bandwith = lr.feature.spectral_bandwidth(y=data, sr=sr)
            rolloff = lr.feature.spectral_rolloff(y=data, sr=sr)
            zero = lr.feature.zero_crossing_rate(y=data)
            harm = lr.effects.harmonic(y=data)
            temp = lr.feature.tempo(y=data)
            mfccmean = np.mean(mfcc,axis=1)
            mfccvar = np.std(mfcc,axis=1)
            stftmean = np.mean(stft)
            stftvar = np.std(stft)
            rmsmean = np.mean(rms)
            rmsvar = np.std(rms)
            centroidmean = np.mean(centroid)
            centroidvar = np.std(centroid)
            bandwithmean = np.mean(bandwith)
            bandwithvar = np.std(bandwith)
            rolloffmean = np.mean(rolloff)
            rolloffvar = np.std(rolloff)
            zeromean = np.mean(zero)
            zerovar = np.std(zero)
            harmmean = np.mean(harm)
            harmvar = np.std(harm)
            prom = list()
            for feat in [mfccmean,mfccvar,stftmean,stftvar,rmsmean,rmsvar,centroidmean,centroidvar,bandwithmean,bandwithvar,rolloffmean,rolloffvar,zeromean,zerovar,harmmean,harmvar,temp[0]]:
                if type(feat) == np.ndarray:
                    prom += feat.tolist()
                else:
                    prom.append(feat)
            prom = np.array(prom)
            ans = np.vstack((prom,ans))
    gtzan_genre[genre] = ans

pop


100it [03:26,  2.06s/it]


metal


100it [03:28,  2.08s/it]


disco


...

country


100it [03:54,  2.34s/it]


jazz


44it [01:38,  2.24s/it]


KeyError: 'duration_ts'

Ошибка, продолжим собирать, используя костыли

In [433]:
ans = np.zeros((55,))
songs = os.listdir(path2+f'/{genre}')
print(genre)
for j,s in tqdm(enumerate(songs)):
    if s == '.DS_Store' or s =='jazz.00054.wav':
        continue
    durts = pydub.utils.mediainfo(path2+f'/{genre}'+f'/{s}')['duration_ts']
    datak, sr = lr.load(path2+f'/{genre}'+f'/{s}',  duration = math.floor(float(durts)))
    for i in range(10):
        data = datak[i*66149:66149*(i+1)]
        mfcc = lr.feature.mfcc(y=data,sr=sr)
        stft = lr.feature.chroma_stft(y=data, sr=sr)
        rms = lr.feature.rms(y=data)
        centroid = lr.feature.spectral_centroid(y=data, sr=sr)
        bandwith = lr.feature.spectral_bandwidth(y=data, sr=sr)
        rolloff = lr.feature.spectral_rolloff(y=data, sr=sr)
        zero = lr.feature.zero_crossing_rate(y=data)
        harm = lr.effects.harmonic(y=data)
        temp = lr.feature.tempo(y=data)
        mfccmean = np.mean(mfcc,axis=1)
        mfccvar = np.std(mfcc,axis=1)
        stftmean = np.mean(stft)
        stftvar = np.std(stft)
        rmsmean = np.mean(rms)
        rmsvar = np.std(rms)
        centroidmean = np.mean(centroid)
        centroidvar = np.std(centroid)
        bandwithmean = np.mean(bandwith)
        bandwithvar = np.std(bandwith)
        rolloffmean = np.mean(rolloff)
        rolloffvar = np.std(rolloff)
        zeromean = np.mean(zero)
        zerovar = np.std(zero)
        harmmean = np.mean(harm)
        harmvar = np.std(harm)
        prom = list()
        for feat in [mfccmean,mfccvar,stftmean,stftvar,rmsmean,rmsvar,centroidmean,centroidvar,bandwithmean,bandwithvar,rolloffmean,rolloffvar,zeromean,zerovar,harmmean,harmvar,temp[0]]:
            if type(feat) == np.ndarray:
                prom += feat.tolist()
            else:
                prom.append(feat)
        prom = np.array(prom)
        ans = np.vstack((prom,ans))
gtzan_genre[genre] = ans

jazz


100it [03:32,  2.12s/it]


#### Датасет из песен Spotify
**Каким образом?** \
Мы используем несколько признаков из библиотеки `librosa.feature`, таких как: MFCC, Chroma STFT, RMS, Spectral bandwith, centroid, rolloff, zero crossing rate, harmonic, tempo. Подробнее об этих методах библиотеки librosa описано в **librosa_preview.ipynb**. Все эти методы на вход принимают предобработанный аудиофайл - временной ряд, который загружается через `librosa.feature.load`. Так как некоторые из этих методов на выходе выдают матрицы(MFCC), а некоторые выдают np.array, то принято решение брать среднее и стандартное отклонение. Для матриц: mean и std по строкам, для векторов: mean и std по единственной строке. Это сделано для того, чтобы во-первых, уменьшить количество признаков, дабы избежать проблемы переобучения(многие из этих методов могут выдавать векторы размерностью до 1000 или более), а во-вторых, чтобы агрегировать данные.\
**Агрегация данных**\
Использование среднего и стандартного отклонения агрегирует информацию, что плохо для нашего случая. Почему? Каждый из этих методов обрабатывает временной ряд так, чтобы на выходе выдать информацию, которая представляет собой либо матрицу, либо вектор. В случае с вектором, каждое его значение будет показывать определенную "черту"(в зависимости от метода) трека в определенный момент времени. Поэтому агрегировать данные по времени не совсем хорошо - мы теряем часть информации и по итогу говорим лишь о том, как в целом выглядит эта "черта" за все время.\
**Борьба с агрегацией** \
Пусть у нас есть трек длительностью, например, 1.5 минуты(90 секунд). Если агрегировать информацию по всему треку сразу, то мы потеряем слишком много информации и собственно, датасет будет содержать в себе так мало данных, что обучиться на них хорошо будет нереально. Поэтому разобьем трек на отрезки по 3 секунды и будем по ним считать librosa.feature и потом по ним среднее и стандартное отклонение. Это позволит агрегировать информацию на меньшем отрезке и терять меньше данных.


In [16]:
genresgt

['pop',
 'metal',
 'disco',
 'blues',
 'reggae',
 'classical',
 'rock',
 'hiphop',
 'country',
 'jazz']

In [14]:
import warnings
warnings.filterwarnings('ignore')
spotify_genre = dict(list(zip(genres,[0 for _ in range(10)])))
for k,genre in enumerate(genresgt):
    genre = genre+'_spotify'
    if genre == 'hiphop_spotify':
        genre = 'hip hop_spotify'
    ans = np.zeros((55,))
    songs = os.listdir(path+f'/{genre}')
    print(genre)
    for j,s in (enumerate(songs)):
        if j == 35:
            break
        if s == '.DS_Store':
            pass
        try:
            dur = pydub.utils.mediainfo(path+f'/{genre}'+f'/{s}')['duration']
            durts = pydub.utils.mediainfo(path+f'/{genre}'+f'/{s}')['duration_ts']
        except:
            pass
        datak, sr = lr.load(path+f'/{genre}'+f'/{s}',  duration = math.floor(float(dur)))
        count = math.floor(float(dur)/3)
        i = 0
        print(count)
        while i <= count:
            data = datak[i*66149:66149*(i+1)]
            mfcc = lr.feature.mfcc(y=data,sr=sr)
            stft = lr.feature.chroma_stft(y=data, sr=sr)
            rms = lr.feature.rms(y=data)
            centroid = lr.feature.spectral_centroid(y=data, sr=sr)
            bandwith = lr.feature.spectral_bandwidth(y=data, sr=sr)
            rolloff = lr.feature.spectral_rolloff(y=data, sr=sr)
            zero = lr.feature.zero_crossing_rate(y=data)
            harm = lr.effects.harmonic(y=data)
            temp = lr.feature.tempo(y=data)
            mfccmean = np.mean(mfcc,axis=1)
            mfccvar = np.std(mfcc,axis=1)
            stftmean = np.mean(stft)
            stftvar = np.std(stft)
            rmsmean = np.mean(rms)
            rmsvar = np.std(rms)
            centroidmean = np.mean(centroid)
            centroidvar = np.std(centroid)
            bandwithmean = np.mean(bandwith)
            bandwithvar = np.std(bandwith)
            rolloffmean = np.mean(rolloff)
            rolloffvar = np.std(rolloff)
            zeromean = np.mean(zero)
            zerovar = np.std(zero)
            harmmean = np.mean(harm)
            harmvar = np.std(harm)
            prom = list()
            for feat in [stftmean,stftvar,rmsmean,rmsvar,centroidmean,centroidvar,bandwithmean,bandwithvar,rolloffmean,rolloffvar,zeromean,zerovar,harmmean,harmvar,temp[0],mfccmean,mfccvar]:
                if type(feat) == np.ndarray:
                    prom += feat.tolist()
                else:
                    prom.append(feat)
            prom = np.array(prom)
            ans = np.vstack((prom,ans))
            i+=1
    ans = np.hstack((ans,np.array([k for i in range(ans.shape[0])]).reshape(-1,1)))
    spotify_genre[genre] = ans

pop_spotify
85
9
41
67
70
73
76
61
72
66
60
40
66
73
77
71
78
76
69
67
69
69
58
66
82
9
66
82
58
70
85
83
75
68
75
metal_spotify
80
96
105
63
81
85
126
88
81
70
58
84
95
80
113
138
67
104
115
86
74
93
84
90
109
97
127
140
71
86
96
131
94
71
79
disco_spotify
78
78
63
74
82
61
81
95
71
68
95
143
82
131
125
87
98
60
80
80
77
58
82
75
68
77
63
111
71
57
67
71
100
95
61
blues_spotify
137
48
56
67
170
78
79
91
115
46
49
87
54
63
57
106
172
63
56
86
39
56
60
57
59
55
59
63
62
56
49
72
55
43
56
reggae_spotify
94
105
79
85
72
74
72
77
71
54
57
98
91
80
76
72
59
69
103
79
88
85
75
50
77
156
77
73
59
92
82
116
59
71
92
classical_spotify
85
76
60
19
104
86
64
89


NoBackendError: 

In [90]:
for k,genre in zip([5,6,7,8,9],['classical','rock','hiphop','country','jazz']):
    genre = genre+'_spotify'
    if genre == 'hiphop_spotify':
        genre = 'hip hop_spotify'
    ans = np.zeros((55,))
    songs = os.listdir(path+f'/{genre}')
    print(genre)
    for j,s in (enumerate(songs)):
        if j == 35:
            break
        if s == '.DS_Store':
            continue
        try:
            dur = pydub.utils.mediainfo(path+f'/{genre}'+f'/{s}')['duration']
            durts = pydub.utils.mediainfo(path+f'/{genre}'+f'/{s}')['duration_ts']
        except:
            continue
        datak, sr = lr.load(path+f'/{genre}'+f'/{s}',  duration = math.floor(float(dur)))
        count = math.floor(float(dur)/3)
        i = 0
        print(count)
        while i <= count:
            data = datak[i*66149:66149*(i+1)]
            mfcc = lr.feature.mfcc(y=data,sr=sr)
            stft = lr.feature.chroma_stft(y=data, sr=sr)
            rms = lr.feature.rms(y=data)
            centroid = lr.feature.spectral_centroid(y=data, sr=sr)
            bandwith = lr.feature.spectral_bandwidth(y=data, sr=sr)
            rolloff = lr.feature.spectral_rolloff(y=data, sr=sr)
            zero = lr.feature.zero_crossing_rate(y=data)
            harm = lr.effects.harmonic(y=data)
            temp = lr.feature.tempo(y=data)
            mfccmean = np.mean(mfcc,axis=1)
            mfccvar = np.std(mfcc,axis=1)
            stftmean = np.mean(stft)
            stftvar = np.std(stft)
            rmsmean = np.mean(rms)
            rmsvar = np.std(rms)
            centroidmean = np.mean(centroid)
            centroidvar = np.std(centroid)
            bandwithmean = np.mean(bandwith)
            bandwithvar = np.std(bandwith)
            rolloffmean = np.mean(rolloff)
            rolloffvar = np.std(rolloff)
            zeromean = np.mean(zero)
            zerovar = np.std(zero)
            harmmean = np.mean(harm)
            harmvar = np.std(harm)
            prom = list()
            for feat in [stftmean,stftvar,rmsmean,rmsvar,centroidmean,centroidvar,bandwithmean,bandwithvar,rolloffmean,rolloffvar,zeromean,zerovar,harmmean,harmvar,temp[0],mfccmean,mfccvar]:
                if type(feat) == np.ndarray:
                    prom += feat.tolist()
                else:
                    prom.append(feat)
            prom = np.array(prom)
            ans = np.vstack((prom,ans))
            i+=1
    ans = np.hstack((ans,np.array([k for i in range(ans.shape[0])]).reshape(-1,1)))
    spotify_genre[genre] = ans

classical_spotify
85
76
60
19
104
86
64
89
56
164
100
43
89
29
115
37
74
238
75
56
59
50
21
53
57
92
82
39
59
119
250
142
47
79
rock_spotify
72
89
76
55
90
62
74
84
88
168
70
66
118
93
54
52
76
90
69
85
59
109
48
135
88
103
74
59
94
83
74
111
77
110
66
hip hop_spotify
95
96
83
53
115
99
91
97
80
78
70
85
68
77
93
73
93
82
134
83
100
66
93
95
86
86
70
62
54
83
78
73
130
75
55
country_spotify
62
68
70
61
73
60
74
60
79
67
73
61
75
83
111
62
50
91
78
62
65
57
97
70
69
74
73
69
75
65
70
60
69
71
64
jazz_spotify
61
91
94
114
63
143
134
122
63
186
63
55
56
199
134
59
96
63
64
45
99
196
55
55
107
91
159
68
166
69
140
71
174
100
67


Собрали данные по жанрам, загрузим их в DataFrame

In [669]:
df = pd.DataFrame(np.vstack((np_genre_spotify.values())))

In [670]:
df = qwer.sort_values(by=55)

На данный момент загрузим "по тупому" train и test классы, дабы избежать лика в данных.

In [671]:
class_train_1 = df.iloc[:1789,:]
class_test_1 = df.iloc[1789:2347,:]
class_train_2 = df.iloc[2347:4651,:]
class_test_2 = df.iloc[4651:4651+988,:]
class_train_3 = df.iloc[5639:7428,:]
class_test_3 = df.iloc[7428:8539,:]
class_train_4 = df.iloc[8539:10328,:]
class_test_4 = df.iloc[10328:11095,:]
class_train_5 = df.iloc[11095:13093,:]
class_test_5 = df.iloc[13093:13949,:]
class_train_6 = df.iloc[13949:15938,:]
class_test_6 = df.iloc[15938:16791,:]
class_train_7 = df.iloc[16791:18860,:]
class_test_7 = df.iloc[18860:19747,:]
class_train_8 = df.iloc[19747:21837,:]
class_test_8 = df.iloc[21837:22733,:]
class_train_9 = df.iloc[22733:24823,:]
class_test_9 = df.iloc[24823:25236,:]
class_train_10 = df.iloc[25236:27726,:]
class_test_10 = df.iloc[27726:28793,:]

In [672]:
test = pd.DataFrame(np.vstack((class_test_1,class_test_2,class_test_3,class_test_4,
                                class_test_5,class_test_6,class_test_7,class_test_8,class_test_9,class_test_10)))
train = pd.DataFrame(np.vstack((class_train_1,class_train_2,class_train_3,
                                 class_train_4,class_test_5,class_train_6,class_train_7,
                                 class_train_8,class_train_9,class_train_10)))
X_train = train.iloc[:,:-1]
y_train = train[55]
X_test = test.iloc[:,:-1]
y_test = test[55]

Сохраним датасет в формате .csv

In [673]:
df.to_csv('spotify.csv')

Проверим на незнакомых данных

In [660]:
for i in os.listdir('Y.Music/blues'):
    e = 'Y.Music/blues/'+i
    dur = pydub.utils.mediainfo(e)['duration']
    durts = pydub.utils.mediainfo(e)['duration_ts']
    datak, sr = lr.load(e,  duration = math.floor(float(dur)))
    count = math.floor(float(dur)/3)
    i = 0
    t = np.zeros((55,))
    print(count)
    while i <= count:
        data = datak[i*66149:66149*(i+1)]
        mfcc = lr.feature.mfcc(y=data,sr=sr)
        stft = lr.feature.chroma_stft(y=data, sr=sr)
        rms = lr.feature.rms(y=data)
        centroid = lr.feature.spectral_centroid(y=data, sr=sr)
        bandwith = lr.feature.spectral_bandwidth(y=data, sr=sr)
        rolloff = lr.feature.spectral_rolloff(y=data, sr=sr)
        try:
            zero = lr.feature.zero_crossing_rate(y=data)
        except:
            break
        harm = lr.effects.harmonic(y=data)
        temp = lr.feature.tempo(y=data)
        mfccmean = np.mean(mfcc,axis=1)
        mfccvar = np.std(mfcc,axis=1)
        stftmean = np.mean(stft)
        stftvar = np.std(stft)
        rmsmean = np.mean(rms)
        rmsvar = np.std(rms)
        centroidmean = np.mean(centroid)
        centroidvar = np.std(centroid)
        bandwithmean = np.mean(bandwith)
        bandwithvar = np.std(bandwith)
        rolloffmean = np.mean(rolloff)
        rolloffvar = np.std(rolloff)
        zeromean = np.mean(zero)
        zerovar = np.std(zero)
        harmmean = np.mean(harm)
        harmvar = np.std(harm)
        prom = list()
        for feat in [stftmean,stftvar,rmsmean,rmsvar,centroidmean,centroidvar,bandwithmean,bandwithvar,rolloffmean,rolloffvar,zeromean,zerovar,harmmean,harmvar,temp[0],mfccmean,mfccvar]:
            if type(feat) == np.ndarray:
                prom += feat.tolist()
            else:
                prom.append(feat)
        prom = np.array(prom)
    #         print(prom.shape)
        t = np.vstack((prom,t))
        i+=1
    print(pd.Series(clf.predict(t).reshape(1,-1)[0]).value_counts())


83
3.0    59
6.0    19
2.0     3
9.0     2
1.0     1
8.0     1
dtype: int64
56
3.0    51
8.0     5
5.0     2
dtype: int64
115
3.0    41
4.0    21
9.0    20
8.0    17
0.0    12
6.0     6
dtype: int64
59
0.0    39
5.0    19
9.0     1
8.0     1
3.0     1
dtype: int64
57


KeyboardInterrupt: 

Это все, конечно, круто. Но ведь лучше расписать это в классы? Сделаем

In [696]:
from typing import DefaultDict
class Dataset:
    def __call__(self, *args, **kwargs):
        raise TypeError("This class is not callable.")
    def make_dataset(self, path, num_songs=35, split=3)->DefaultDict(list):
        def __init__(self, path, num_songs=35, split=3):
            self.num_songs = num_songs
            self.split = split
            self.path = path
        songs_dict = dict(list(zip(genres,[0 for _ in range(len(genres))])))
        genresgt = os.listdir(path)
        for k,genre in tqdm(enumerate(genresgt)):
            ans = np.zeros((55,))
            songs = os.listdir(path+f'/{genre}')
            for j,s in (enumerate(songs)):
                if j == num_songs:
                    break
                try:
                    dur = pydub.utils.mediainfo(path+f'/{genre}'+f'/{s}')['duration']
                    durts = pydub.utils.mediainfo(path+f'/{genre}'+f'/{s}')['duration_ts']
                except:
                    pass
                datak, sr = lr.load(path+f'/{genre}'+f'/{s}',  duration = math.floor(float(dur)))
                count = math.floor(float(dur)/split)
                i = 0
                while i <= count:
                    data = datak[i*66149:66149*(i+1)]
                    mfcc = lr.feature.mfcc(y=data,sr=sr)
                    stft = lr.feature.chroma_stft(y=data, sr=sr)
                    rms = lr.feature.rms(y=data)
                    centroid = lr.feature.spectral_centroid(y=data, sr=sr)
                    bandwith = lr.feature.spectral_bandwidth(y=data, sr=sr)
                    rolloff = lr.feature.spectral_rolloff(y=data, sr=sr)
                    zero = lr.feature.zero_crossing_rate(y=data)
                    harm = lr.effects.harmonic(y=data)
                    temp = lr.feature.tempo(y=data)
                    mfccmean = np.mean(mfcc,axis=1)
                    mfccvar = np.std(mfcc,axis=1)
                    stftmean = np.mean(stft)
                    stftvar = np.std(stft)
                    rmsmean = np.mean(rms)
                    rmsvar = np.std(rms)
                    centroidmean = np.mean(centroid)
                    centroidvar = np.std(centroid)
                    bandwithmean = np.mean(bandwith)
                    bandwithvar = np.std(bandwith)
                    rolloffmean = np.mean(rolloff)
                    rolloffvar = np.std(rolloff)
                    zeromean = np.mean(zero)
                    zerovar = np.std(zero)
                    harmmean = np.mean(harm)
                    harmvar = np.std(harm)
                    prom = list()
                    for feat in [stftmean,stftvar,rmsmean,rmsvar,centroidmean,centroidvar,bandwithmean,bandwithvar,rolloffmean,rolloffvar,zeromean,zerovar,harmmean,harmvar,temp[0],mfccmean,mfccvar]:
                        if type(feat) == np.ndarray:
                            prom += feat.tolist()
                        else:
                            prom.append(feat)
                    prom = np.array(prom)
                    ans = np.vstack((prom,ans))
                    i+=1
            ans = np.hstack((ans,np.array([k for i in range(ans.shape[0])]).reshape(-1,1)))
            songs_dict[genre] = ans
        return songs_dict
    def get_song_features(self,path, split=3):
        e = path
        dur = pydub.utils.mediainfo(e)['duration']
        durts = pydub.utils.mediainfo(e)['duration_ts']
        datak, sr = lr.load(e,  duration = math.floor(float(dur)))
        count = math.floor(float(dur)/split)
        i = 0
        t = np.zeros((55,))
        print(count)
        while i <= count:
            data = datak[i*66149:66149*(i+1)]
            mfcc = lr.feature.mfcc(y=data,sr=sr)
            stft = lr.feature.chroma_stft(y=data, sr=sr)
            rms = lr.feature.rms(y=data)
            centroid = lr.feature.spectral_centroid(y=data, sr=sr)
            bandwith = lr.feature.spectral_bandwidth(y=data, sr=sr)
            rolloff = lr.feature.spectral_rolloff(y=data, sr=sr)
            try:
                zero = lr.feature.zero_crossing_rate(y=data)
            except:
                break
            harm = lr.effects.harmonic(y=data)
            temp = lr.feature.tempo(y=data)
            mfccmean = np.mean(mfcc,axis=1)
            mfccvar = np.std(mfcc,axis=1)
            stftmean = np.mean(stft)
            stftvar = np.std(stft)
            rmsmean = np.mean(rms)
            rmsvar = np.std(rms)
            centroidmean = np.mean(centroid)
            centroidvar = np.std(centroid)
            bandwithmean = np.mean(bandwith)
            bandwithvar = np.std(bandwith)
            rolloffmean = np.mean(rolloff)
            rolloffvar = np.std(rolloff)
            zeromean = np.mean(zero)
            zerovar = np.std(zero)
            harmmean = np.mean(harm)
            harmvar = np.std(harm)
            prom = list()
            for feat in [stftmean,stftvar,rmsmean,rmsvar,centroidmean,centroidvar,bandwithmean,bandwithvar,rolloffmean,rolloffvar,zeromean,zerovar,harmmean,harmvar,temp[0],mfccmean,mfccvar]:
                if type(feat) == np.ndarray:
                    prom += feat.tolist()
                else:
                    prom.append(feat)
            prom = np.array(prom)
            t = np.vstack((prom,t))
            i+=1
        return t
